# 3r0r

In [1]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

In [2]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines) - 1
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i+1][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i+1][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i+1][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [3]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [4]:
nrun = 1000

1000

In [5]:
for irun = 1:nrun

    if irun % 100 == 0
        print("irun = $irun")
    end
    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:1594]
        t2 = t[:, 1595:end]
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

irun = 100irun = 200irun = 300irun = 400irun = 500irun = 600irun = 700irun = 800irun = 900irun = 1000

In [6]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

Row,fnames,score,omega,phi,prou,d,struc
,Any,Any,Any,Any,Any,Any,Any
1,1/complex.1.pdb,2438.18,"[-0.131239, -0.0738241, -0.988598]",176.45,"[2.287, -3.12306, -0.0703888]",0.215519,"1x3188 TrjArray{Float64, Int64}\n| 7.64 -8.73 61.51 | … 22.36 16.27 88.98 |"
2,1/complex.2.pdb,2019.56,"[0.148473, -0.0434833, 0.98796]",177.204,"[0.339791, 5.53982, 0.192761]",-0.337493,"1x3188 TrjArray{Float64, Int64}\n| 7.64 -8.73 61.51 | … 23.16 10.92 91.13 |"
3,1/complex.3.pdb,1977.51,"[-0.123651, -0.991774, 0.0330964]",130.459,"[11.0909, 0.900187, 68.4116]",6.61822,"1x3188 TrjArray{Float64, Int64}\n| 7.64 -8.73 61.51 | … -6.89 -14.94 49.86 |"
4,1/complex.4.pdb,1938.64,"[0.320035, -0.0694202, 0.944859]",176.962,"[-10.4778, 25.9029, 5.45206]",0.956278,"1x3188 TrjArray{Float64, Int64}\n| 7.64 -8.73 61.51 | … 33.01 46.83 89.06 |"
5,1/complex.5.pdb,1927.28,"[-0.0410833, 0.854917, 0.517135]",173.882,"[13.5648, -31.0287, 52.3737]",-0.322915,"1x3188 TrjArray{Float64, Int64}\n| 7.64 -8.73 61.51 | … 20.58 15.54 54.29 |"
6,1/complex.6.pdb,1892.77,"[-0.967024, 0.0406925, 0.251414]",163.8,"[19.1629, 19.823, 70.4985]",-26.9341,"1x3188 TrjArray{Float64, Int64}\n| 7.64 -8.73 61.51 | … 26.98 52.41 58.33 |"
7,1/complex.7.pdb,1860.98,"[0.993216, -0.112046, 0.0311206]",161.789,"[0.37101, 21.7032, 66.2993]",-25.3667,"1x3188 TrjArray{Float64, Int64}\n| 7.64 -8.73 61.51 | … -12.19 42.74 31.32 |"
8,1/complex.8.pdb,1854.27,"[-0.0461843, 0.940935, -0.335422]",129.447,"[10.2844, 26.0568, 71.679]",-17.4496,"1x3188 TrjArray{Float64, Int64}\n| 7.64 -8.73 61.51 | … 23.77 -27.18 90.04 |"
9,1/complex.9.pdb,1854.04,"[0.718009, -0.401641, 0.56846]",176.099,"[-24.343, 16.0746, 42.1044]",-0.486495,"1x3188 TrjArray{Float64, Int64}\n| 7.64 -8.73 61.51 | … 31.62 -9.16 59.05 |"


In [7]:
df_org = deepcopy(df);

In [8]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [9]:
#using BSON: @save, @load
#@save "df.bson" df

In [10]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)